In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Deeksha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# scraper.py
import requests
from bs4 import BeautifulSoup
import time
import random

def fetch_page(url: str) -> BeautifulSoup:
    """Fetch a page and return parsed HTML."""
    print(f"📥 Fetching: {url}")
    headers = {'User-Agent': 'Mozilla/5.0 (Educational Scraper)'}
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        resp.raise_for_status()
        return BeautifulSoup(resp.text, 'html.parser')
    except Exception as e:
        print(f"❌ Error fetching {url}: {e}")
        return None

def extract_quotes_from_page(soup: BeautifulSoup) -> list:
    """Extract quotes and authors from quotes.toscrape.com."""
    quotes = []
    for quote_div in soup.select('.quote'):
        text = quote_div.select_one('.text').get_text(strip=True)
        author = quote_div.select_one('.author').get_text(strip=True)
        tags = [t.get_text(strip=True) for t in quote_div.select('.tag')]
        quotes.append({
            'text': text,
            'author': author,
            'tags': tags
        })
    return quotes

def scrape_quotes_toscrape(num_pages: int = 3) -> list:
    """Scrape multiple pages from quotes.toscrape.com."""
    all_quotes = []
    base_url = "https://quotes.toscrape.com/page/{}/"
    
    for page in range(1, num_pages + 1):
        url = base_url.format(page)
        soup = fetch_page(url)
        if soup:
            quotes = extract_quotes_from_page(soup)
            all_quotes.extend(quotes)
            print(f"  ➕ Got {len(quotes)} quotes (total: {len(all_quotes)})")
        
        # Be polite: pause between requests
        time.sleep(random.uniform(0.5, 1.5))
    
    return all_quotes

In [3]:
# cleaner.py
import nltk
from nltk.corpus import stopwords
import re

# Download once (or ensure it's in setup)
nltk.download('stopwords', quiet=True)
STOP_WORDS = set(stopwords.words('english'))

def clean_text(text: str) -> str:
    """Clean text for NLP: lowercase, remove punctuation, strip stopwords."""
    # Lowercase
    text = text.lower()
    
    # Keep only letters and spaces
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenize
    words = text.split()
    
    # Remove stopwords
    words = [w for w in words if w not in STOP_WORDS and len(w) > 2]
    
    return ' '.join(words)

def clean_quotes(quotes: list) -> list:
    """Apply cleaning to a list of quote dicts."""
    print("🪥 Cleaning text...")
    cleaned = []
    for q in quotes:
        cleaned_text = clean_text(q['text'])
        if cleaned_text:  # skip empty
            cleaned.append({
                'original_text': q['text'],
                'cleaned_text': cleaned_text,
                'author': q['author'],
                'tags': q['tags']
            })
    print(f"✅ Cleaned {len(cleaned)} quotes")
    return cleaned

In [5]:
# Install if needed (run once)
!pip install requests beautifulsoup4 nltk

import requests
from bs4 import BeautifulSoup
import nltk
import json
import time
import random
import os
import re

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Download stopwords (safe to run multiple times)
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

from nltk.corpus import stopwords
STOP_WORDS = set(stopwords.words('english'))

In [7]:
def fetch_page(url: str) -> BeautifulSoup:
    print(f"📥 Fetching: {url}")
    headers = {'User-Agent': 'Mozilla/5.0 (Educational Scraper)'}
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        resp.raise_for_status()
        return BeautifulSoup(resp.text, 'html.parser')
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

def extract_quotes_from_page(soup: BeautifulSoup) -> list:
    quotes = []
    for quote_div in soup.select('.quote'):
        text = quote_div.select_one('.text').get_text(strip=True)
        author = quote_div.select_one('.author').get_text(strip=True)
        tags = [t.get_text(strip=True) for t in quote_div.select('.tag')]
        quotes.append({'text': text, 'author': author, 'tags': tags})
    return quotes

def scrape_quotes_toscrape(num_pages: int = 2) -> list:
    all_quotes = []
    base_url = "https://quotes.toscrape.com/page/{}/"
    for page in range(1, num_pages + 1):
        url = base_url.format(page)
        soup = fetch_page(url)
        if soup:
            quotes = extract_quotes_from_page(soup)
            all_quotes.extend(quotes)
            print(f"  ➕ Got {len(quotes)} quotes")
        time.sleep(1)  # be polite
    return all_quotes

In [8]:
def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # keep only letters/spaces
    words = [w for w in text.split() if w not in STOP_WORDS and len(w) > 2]
    return ' '.join(words)

def clean_quotes(quotes: list) -> list:
    print("🪥 Cleaning text...")
    cleaned = []
    for q in quotes:
        cleaned_text = clean_text(q['text'])
        if cleaned_text:
            cleaned.append({
                'original_text': q['text'],
                'cleaned_text': cleaned_text,
                'author': q['author'],
                'tags': q['tags']
            })
    print(f"✅ Cleaned {len(cleaned)} quotes")
    return cleaned

In [9]:
def save_to_json(data: list, filepath: str):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"✅ Saved to {filepath}")

# Run full pipeline
print("🚀 Starting Project 2 ETL...")
raw_data = scrape_quotes_toscrape(num_pages=2)

if raw_data:
    cleaned_data = clean_quotes(raw_data)
    save_to_json(cleaned_data, "output/cleaned_quotes.json")
    print("\n🎉 Done! Check the 'output' folder.")
else:
    print("⚠️ No data scraped.")

🚀 Starting Project 2 ETL...
📥 Fetching: https://quotes.toscrape.com/page/1/
  ➕ Got 10 quotes
📥 Fetching: https://quotes.toscrape.com/page/2/
  ➕ Got 10 quotes
🪥 Cleaning text...
✅ Cleaned 20 quotes
✅ Saved to output/cleaned_quotes.json

🎉 Done! Check the 'output' folder.


In [10]:
# Load and display first 2 cleaned quotes
with open("output/cleaned_quotes.json", "r", encoding="utf-8") as f:
    data = json.load(f)

for i, quote in enumerate(data[:2]):
    print(f"\n--- Quote {i+1} ---")
    print("Original:", quote["original_text"])
    print("Cleaned: ", quote["cleaned_text"])
    print("Author:  ", quote["author"])


--- Quote 1 ---
Original: “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Cleaned:  world created process thinking cannot changed without changing thinking
Author:   Albert Einstein

--- Quote 2 ---
Original: “It is our choices, Harry, that show what we truly are, far more than our abilities.”
Cleaned:  choices harry show truly far abilities
Author:   J.K. Rowling
